In [1]:
import tensorflow as tf
import numpy as np
import time

In [2]:
read_file_data = np.loadtxt('diabetes_pidd.csv', delimiter=',', dtype=np.float32)
x_data = read_file_data[:, 0:-1]
y_data = read_file_data[:,[-1]]

In [3]:
print(x_data)
print("\n")
print(y_data)

[[   6.          148.           72.         ...,   33.59999847
     0.62699997   50.        ]
 [   1.           85.           66.         ...,   26.60000038
     0.35100001   31.        ]
 [   8.          183.           64.         ...,   23.29999924
     0.67199999   32.        ]
 ..., 
 [   5.          121.           72.         ...,   26.20000076    0.245
    30.        ]
 [   1.          126.           60.         ...,   30.10000038
     0.34900001   47.        ]
 [   1.           93.           70.         ...,   30.39999962    0.315
    23.        ]]


[[ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [

In [4]:
def MinMaxScaler(data):
    numerator = data - np.min(data,0)
    denominator = np.max(data, 0) - np.min(data,0)
    return numerator / (denominator + 1e-7)

In [5]:
Normalizaion_x = MinMaxScaler(x_data)
print(Normalizaion_x)

[[ 0.35294119  0.74371856  0.59016395 ...,  0.50074512  0.23441501
   0.48333332]
 [ 0.05882353  0.42713568  0.54098362 ...,  0.39642325  0.11656704
   0.16666667]
 [ 0.47058824  0.91959798  0.52459013 ...,  0.34724292  0.25362936
   0.18333334]
 ..., 
 [ 0.29411766  0.60804021  0.59016395 ...,  0.39046201  0.07130657
   0.15000001]
 [ 0.05882353  0.63316584  0.49180329 ...,  0.44858423  0.11571307
   0.43333334]
 [ 0.05882353  0.46733668  0.57377046 ...,  0.45305514  0.10119555
   0.03333334]]


In [6]:
X = tf.placeholder(tf.float32, shape=[None,8])
Y = tf.placeholder(tf.float32, shape=[None,1])

In [7]:
with tf.name_scope("model"):
    W = tf.Variable(tf.random_normal([8, 1]),name='weight')
    b = tf.Variable(tf.random_normal([1]), name='bias')
    hypothesis = tf.sigmoid(tf.matmul(X,W) + b)

    w1_hist = tf.summary.histogram("weights1", W)
    b1_hist = tf.summary.histogram("biases1", b)
    hypothesis_hist = tf.summary.histogram("layer1", hypothesis)

In [8]:
with tf.name_scope("cost"):
    cost = - tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

In [9]:
with tf.name_scope("train"):
    train = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [10]:
predicted = tf.cast(hypothesis > 0.5 , dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype= tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)

In [11]:
init = tf.global_variables_initializer()

In [12]:
start = time.time()  
with tf.Session() as sess:
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/log001")
    writer.add_graph(sess.graph)
    sess.run(init)
    for step in range(1001):
        summary, _ = sess.run([merged_summary, train], feed_dict={X: Normalizaion_x, Y: y_data})
        writer.add_summary(summary, global_step=step)
        cost_value, _ = sess.run([cost, train], feed_dict = {X:Normalizaion_x, Y:y_data})

        if step % 100 == 0:
            print(step, cost_value)
    h, c, a = sess.run([hypothesis,predicted,accuracy], feed_dict={X:Normalizaion_x, Y:y_data})
    print("\nHypothesis : ", h,"\nCorrect : ", c , "\nAccuracy! : ", a)
end = time.time() - start

0 0.993547
100 0.644634
200 0.595313
300 0.563991
400 0.540838
500 0.522926
600 0.509203
700 0.498824
800 0.491038
900 0.485234
1000 0.480943

Hypothesis :  [[ 0.66700864]
 [ 0.09010738]
 [ 0.73597825]
 [ 0.07627917]
 [ 0.84673405]
 [ 0.18924417]
 [ 0.11673506]
 [ 0.63017166]
 [ 0.71883202]
 [ 0.10381507]
 [ 0.20846267]
 [ 0.80388314]
 [ 0.74991167]
 [ 0.70935774]
 [ 0.63130093]
 [ 0.47014883]
 [ 0.33835325]
 [ 0.22902973]
 [ 0.36795989]
 [ 0.26278466]
 [ 0.37113184]
 [ 0.32906348]
 [ 0.85360175]
 [ 0.31479329]
 [ 0.64212877]
 [ 0.46096441]
 [ 0.6326673 ]
 [ 0.0902402 ]
 [ 0.5802716 ]
 [ 0.2725426 ]
 [ 0.43926433]
 [ 0.54676116]
 [ 0.09176701]
 [ 0.06979801]
 [ 0.45662001]
 [ 0.24306445]
 [ 0.59080952]
 [ 0.41677985]
 [ 0.19571356]
 [ 0.59023255]
 [ 0.64916211]
 [ 0.58538616]
 [ 0.17537175]
 [ 0.84892273]
 [ 0.58928746]
 [ 0.88289118]
 [ 0.41799524]
 [ 0.07327597]
 [ 0.36278576]
 [ 0.13232628]
 [ 0.0730281 ]
 [ 0.14373647]
 [ 0.11971503]
 [ 0.78276736]
 [ 0.69245422]
 [ 0.05295155]
 [ 

In [13]:
print(end," estimation second")

2.693279266357422  estimation second
